In [1]:
import os
os.chdir("../")
%pwd

'd:\\Final-Year-Project\\Credit-Card-Fraud-Detection-Using-GNN'

In [3]:
# Entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [4]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml, create_directories

In [5]:
# Configuration

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [6]:
import os
import numpy as np
import pandas as pd
from Credit_Card_Fraud_Detection import logger
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [7]:
# Components

class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.label_encoders = {}
        self.scaler = StandardScaler()

    def load_data(self):
        try:
            df = pd.read_csv(self.config.data_path)
            logger.info("Data loaded successfully.")
            return df
        except FileNotFoundError:
            logger.error(f"File not found: {self.config.data_path}")
            return None

    def drop_unwanted_columns(self, df):
        df = df.drop(columns=["merch_zipcode", "Unnamed: 0"], errors="ignore")
        logger.info("Unwanted columns dropped.")
        return df

    def handle_missing_categorical(self, df):
        df["category"] = df["category"].fillna("unknown")
        df["state"] = df["state"].fillna("unknown")
        logger.info("Missing categorical values handled.")
        return df

    def encode_categorical(self, df):
        if "category" not in self.label_encoders:
            self.label_encoders["category"] = LabelEncoder()
            df["category"] = self.label_encoders["category"].fit_transform(df["category"])
        else:
            df["category"] = self.label_encoders["category"].transform(df["category"])

        if "state" not in self.label_encoders:
            self.label_encoders["state"] = LabelEncoder()
            df["state"] = self.label_encoders["state"].fit_transform(df["state"])
        else:
            df["state"] = self.label_encoders["state"].transform(df["state"])

        logger.info("Categorical columns encoded.")
        return df

    def handle_numerical_features(self, df):
        num_features = ["amt", "city_pop", "unix_time", "lat", "long", "merch_lat", "merch_long"]
        existing_features = [col for col in num_features if col in df.columns]

        df[existing_features] = df[existing_features].apply(pd.to_numeric, errors='coerce').fillna(0)
        logger.info("Numerical features converted and missing values handled.")
        return df

    def normalize_numerical_features(self, df):
        num_features = ["amt", "city_pop", "unix_time", "lat", "long", "merch_lat", "merch_long"]
        existing_features = [col for col in num_features if col in df.columns]

        df[existing_features] = self.scaler.fit_transform(df[existing_features])
        logger.info("Numerical features normalized.")
        return df

    def preprocess(self):
        df = self.load_data()
        if df is None:
            return

        df = self.drop_unwanted_columns(df)
        df = self.handle_missing_categorical(df)
        df = self.encode_categorical(df)
        df = self.handle_numerical_features(df)
        df = self.normalize_numerical_features(df)

        # Save preprocessed dataset
        preprocessed_path = os.path.join(self.config.root_dir, "transformed_dataset.csv")
        df.to_csv(preprocessed_path, index=False)
        logger.info(f"Preprocessed data saved at {preprocessed_path}")

        return df # return the dataframe for further processing

In [8]:
# Pipeline

try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    transformed_data = data_transformation.preprocess()  # Corrected method call
except Exception as e:
    raise e  # This will re-raise the exception if any error occurs

[2025-03-17 23:59:09,060: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-17 23:59:09,062: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-17 23:59:09,063: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-17 23:59:09,063: INFO: common: created directory at: artifacts]
[2025-03-17 23:59:09,063: INFO: common: created directory at: artifacts/data_transformation]


[2025-03-17 23:59:12,860: INFO: 1399541205: Data loaded successfully.]
[2025-03-17 23:59:12,960: INFO: 1399541205: Unwanted columns dropped.]
[2025-03-17 23:59:13,085: INFO: 1399541205: Missing categorical values handled.]
[2025-03-17 23:59:13,345: INFO: 1399541205: Categorical columns encoded.]
[2025-03-17 23:59:13,427: INFO: 1399541205: Numerical features converted and missing values handled.]
[2025-03-17 23:59:13,594: INFO: 1399541205: Numerical features normalized.]
[2025-03-17 23:59:26,703: INFO: 1399541205: Preprocessed data saved at artifacts/data_transformation\transformed_dataset.csv]
